In [152]:
import time
start_time = time.time()
import numpy as np
import pandas as pd
import nltk, re, pprint
from __future__ import division

In [153]:
# read the training data
df_train = pd.read_csv("/Users/shijiabian/Dropbox/Duke Statistics Courses/Spring 2016 Duke Course/STA 571 Adv ML/Final Project/Data/train.csv", encoding="ISO-8859-1") #update here
df_test = pd.read_csv("/Users/shijiabian/Dropbox/Duke Statistics Courses/Spring 2016 Duke Course/STA 571 Adv ML/Final Project/Data/test.csv", encoding="ISO-8859-1") #update here
df_att = pd.read_csv("/Users/shijiabian/Dropbox/Duke Statistics Courses/Spring 2016 Duke Course/STA 571 Adv ML/Final Project/Data/attributes.csv", encoding="ISO-8859-1") #update here
df_pro = pd.read_csv("/Users/shijiabian/Dropbox/Duke Statistics Courses/Spring 2016 Duke Course/STA 571 Adv ML/Final Project/Data/product_descriptions.csv", encoding="ISO-8859-1") #update here

In [154]:
df_train_df = pd.DataFrame(df_train)
df_train_pro = pd.DataFrame(df_pro)
df_test_df = pd.DataFrame(df_test)

In [155]:
result_1 = pd.merge(df_train_df, df_train_pro, on='product_uid')
result_test_1 = pd.merge(df_test_df, df_train_pro, on='product_uid')

In [289]:
# attr: combine t
df_att["name_and_value"] = df_att["name"].map(str) + ' ' + df_att["value"]

In [290]:
df_att.head(2)

,product_uid,name,value,name_and_value
0,100001,Bullet01,Versatile connector for various 90Â° connectio...,Bullet01 Versatile connector for various 90Â° ...
1,100001,Bullet02,Stronger than angled nailing or screw fastenin...,Bullet02 Stronger than angled nailing or screw...


In [157]:
# further handle the attribute data
def f(x):
    return x['name_and_value'].values

df_att_2 = df_att.groupby('product_uid').apply(f)

#series.name = 'metric'
df_att_2 = df_att_2.reset_index()
df_att_2.columns = ['product_uid', 'name_and_value']

In [295]:
df_att_2[:1]['name_and_value']

0    [Bullet01 Versatile connector for various 90Â°...
Name: name_and_value, dtype: object

In [158]:
# merge the new attribute table 
result = pd.merge(result_1, df_att_2, on='product_uid')
result_test = pd.merge(result_test_1, df_att_2, on='product_uid')

In [159]:
# select all rows that the search_term has bracket
#result[result['search_term'].str.contains("bracket")]
#dataframe.groupby(['date', 'sitename', 'name']).sum()
result.head(5)

,id,product_uid,product_title,search_term,relevance,product_description,name_and_value
0,2,100001,Simpson Strong-Tie 12-Gauge Angle,angle bracket,3.00,"Not only do angles make joints stronger, they ...",[Bullet01 Versatile connector for various 90Â°...
1,3,100001,Simpson Strong-Tie 12-Gauge Angle,l bracket,2.50,"Not only do angles make joints stronger, they ...",[Bullet01 Versatile connector for various 90Â°...
2,9,100002,BEHR Premium Textured DeckOver 1-gal. #SC-141 ...,deck over,3.00,BEHR Premium Textured DECKOVER is an innovativ...,"[Application Method Brush,Roller,Spray, Assemb..."
3,16,100005,Delta Vero 1-Handle Shower Only Faucet Trim Ki...,rain shower head,2.33,Update your bathroom with the Delta Vero Singl...,"[Bath Faucet Type Combo Tub and Shower, Built-..."
4,17,100005,Delta Vero 1-Handle Shower Only Faucet Trim Ki...,shower only faucet,2.67,Update your bathroom with the Delta Vero Singl...,"[Bath Faucet Type Combo Tub and Shower, Built-..."


In [160]:
# define the function for splitting the words
def split_search(value) :
    res = [w.lower() for w in (str(value).split())]
    return res

In [161]:
# add a new column called search_split
ss = result["search_term"].apply(split_search) 
result['search_split'] = pd.Series(ss, index=result.index)
ss = result_test["search_term"].apply(split_search) 
result_test['search_split'] = pd.Series(ss, index=result_test.index)

In [162]:
result.head(4)

,id,product_uid,product_title,search_term,relevance,product_description,name_and_value,search_split
0,2,100001,Simpson Strong-Tie 12-Gauge Angle,angle bracket,3.00,"Not only do angles make joints stronger, they ...",[Bullet01 Versatile connector for various 90Â°...,"[angle, bracket]"
1,3,100001,Simpson Strong-Tie 12-Gauge Angle,l bracket,2.50,"Not only do angles make joints stronger, they ...",[Bullet01 Versatile connector for various 90Â°...,"[l, bracket]"
2,9,100002,BEHR Premium Textured DeckOver 1-gal. #SC-141 ...,deck over,3.00,BEHR Premium Textured DECKOVER is an innovativ...,"[Application Method Brush,Roller,Spray, Assemb...","[deck, over]"
3,16,100005,Delta Vero 1-Handle Shower Only Faucet Trim Ki...,rain shower head,2.33,Update your bathroom with the Delta Vero Singl...,"[Bath Faucet Type Combo Tub and Shower, Built-...","[rain, shower, head]"


In [252]:
# define the function for extracting the number from the value_and_name
def array_combine(ind) :
    val = ''.join(str(result['name_and_value'][ind]))
    return val
l=[]
for i in range(result.shape[0]):
    ind = array_combine(i)
    l.append(ind)
result['name_and_value_bind'] = pd.Series(l, index=result.index)

# define the function for extracting the number from the value_and_name
def array_combine_test(ind) :
    val = ''.join(str(result_test['name_and_value'][ind]))
    return val
l_test=[]
for i in range(result_test.shape[0]):
    ind = array_combine_test(i)
    l_test.append(ind)
result_test['name_and_value_bind'] = pd.Series(l_test, index=result_test.index)

In [255]:
# help to split the string
def split_search(value) :
    res = [w.lower() for w in (str(value).split())]
    return res

# calculate the word length for each of the searching words
# remove the word that has length 1
nrow = result.shape[0]
new_search_split = [0]*nrow
for index, row in result.iterrows():
    each_word_len = [len(w) for w in row["search_split"]]
    row["search_split"] = [word for word in row["search_split"] if len(word) > 1]
    new_search_split[index] = row["search_split"]
    each_word_len = [mm for mm in each_word_len if mm > 1]

# add a new column called search_split
result['new_search_split'] = pd.Series(new_search_split, index=result.index)

# help to split the string for test data
def split_search(value) :
    res = [w.lower() for w in (str(value).split())]
    return res

# calculate the word length for each of the searching words
# remove the word that has length 1

nrow = result_test.shape[0]
new_search_split = [0]*nrow
for index, row in result_test.iterrows():
    each_word_len = [len(str(w)) for w in row["search_split"]]
    row["search_split"] = [word for word in row["search_split"] if len(word) > 1]
    new_search_split[index] = row["search_split"]
    each_word_len = [mm for mm in each_word_len if mm > 1]

# add a new column called search_split
result_test['new_search_split'] = pd.Series(new_search_split, index=result_test.index)

In [256]:
# find how many words in the product description match the searching query 
nrow = result.shape[0]
desc_match_word = [0]*nrow
for index, row in result.iterrows():
    ll = [len(re.findall(w, row["product_description"])) for w in row["new_search_split"]]
    desc_match_word[index] = sum(ll)

# add a new column called description match words
result['desc_match_word'] = pd.Series(desc_match_word, index=result.index)


In [257]:
import sys
reload(sys)
sys.setdefaultencoding("utf-8")

In [258]:
#result_test["product_description"][73024] = result_test["product_description"][1]
#result_test["new_search_split"][73024] = result_test["new_search_split"][1]
#118900
# 118901
# find how many words in the product description match the searching query 
nrow = result_test.shape[0]
desc_match_word = [0]*nrow
for index, row in result_test.iterrows():    
    ll = [len(re.findall(w, row["product_description"])) for w in row["new_search_split"]]
    print (ll)
    desc_match_word[index] = sum(ll)
result_test['desc_match_word'] = pd.Series(desc_match_word, index=result_test.index)


In [259]:
import sys
reload(sys)
sys.setdefaultencoding("utf-8")

In [260]:
# find how many words match in the product title
nrow = result.shape[0]
title_match_word = [0]*nrow
for index, row in result.iterrows():
    ll = [len(re.findall(w, str(row["product_title"]).lower())) for w in row["new_search_split"]]
    print (ll)
    print(index)
    title_match_word[index] = sum(ll)

# add a new column called description match words
result['title_match_word'] = pd.Series(title_match_word, index=result.index)

# test data
# find how many words match in the product title
nrow = result_test.shape[0]
title_match_word = [0]*nrow
for index, row in result_test.iterrows():
    ll = [len(re.findall(w, str(row["product_title"]).lower())) for w in row["new_search_split"]]
    print (ll)
    print(index)
    title_match_word[index] = sum(ll)

# add a new column called description match words
result_test['title_match_word'] = pd.Series(title_match_word, index=result_test.index)


In [286]:
Regex_Pattern = r"\d+"
Test_String = result['name_and_value_bind'][1]
match = re.findall(Regex_Pattern, Test_String)
stringTry = 'i am a 90'
[re.findall(w, str(stringTry)) for w in match]
#re.findall(w, str(stringTry))

[[],
 ['90'],
 [],
 ['0'],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 ['9'],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 ['0'],
 [],
 []]

In [288]:
result.head(1)

,id,product_uid,product_title,search_term,relevance,product_description,name_and_value,search_split,name_and_value_bind,new_search_split,desc_match_word,title_match_word
0,2,100001,Simpson Strong-Tie 12-Gauge Angle,angle bracket,3,"Not only do angles make joints stronger, they ...",[Bullet01 Versatile connector for various 90Â°...,"[angle, bracket]",[ u'Bullet01 Versatile connector for various 9...,"[angle, bracket]",3,1


In [ ]:
feature = ['Pclass','Sex','Age','Fare','Parch']

In [274]:
result.head(5)

,id,product_uid,product_title,search_term,relevance,product_description,name_and_value,search_split,name_and_value_bind,new_search_split,desc_match_word,title_match_word
0,2,100001,Simpson Strong-Tie 12-Gauge Angle,angle bracket,3.00,"Not only do angles make joints stronger, they ...",[Bullet01 Versatile connector for various 90Â°...,"[angle, bracket]",[ u'Bullet01 Versatile connector for various 9...,"[angle, bracket]",3,1
1,3,100001,Simpson Strong-Tie 12-Gauge Angle,l bracket,2.50,"Not only do angles make joints stronger, they ...",[Bullet01 Versatile connector for various 90Â°...,"[l, bracket]",[ u'Bullet01 Versatile connector for various 9...,[bracket],0,0
2,9,100002,BEHR Premium Textured DeckOver 1-gal. #SC-141 ...,deck over,3.00,BEHR Premium Textured DECKOVER is an innovativ...,"[Application Method Brush,Roller,Spray, Assemb...","[deck, over]","[u'Application Method Brush,Roller,Spray' u'As...","[deck, over]",4,2
3,16,100005,Delta Vero 1-Handle Shower Only Faucet Trim Ki...,rain shower head,2.33,Update your bathroom with the Delta Vero Singl...,"[Bath Faucet Type Combo Tub and Shower, Built-...","[rain, shower, head]",[u'Bath Faucet Type Combo Tub and Shower' u'Bu...,"[rain, shower, head]",0,1
4,17,100005,Delta Vero 1-Handle Shower Only Faucet Trim Ki...,shower only faucet,2.67,Update your bathroom with the Delta Vero Singl...,"[Bath Faucet Type Combo Tub and Shower, Built-...","[shower, only, faucet]",[u'Bath Faucet Type Combo Tub and Shower' u'Bu...,"[shower, only, faucet]",1,3


In [261]:
# build random forest model

from sklearn.ensemble import RandomForestClassifier
cols = ['desc_match_word', 'title_match_word'] 
colsRes = ['relevance']
trainArr = result.as_matrix(cols) #training array
trainRes = result.as_matrix(colsRes) # training results
rf = RandomForestClassifier(n_estimators=100) # initialize
rf.fit(trainArr, trainRes) # fit the data to the algorithm


RandomForestClassifier(bootstrap=True, compute_importances=None,
            criterion='gini', max_depth=None, max_features='auto',
            max_leaf_nodes=None, min_density=None, min_samples_leaf=1,
            min_samples_split=2, n_estimators=100, n_jobs=1,
            oob_score=False, random_state=None, verbose=0)

In [14]:
testArr = result_test.as_matrix(cols)
results = rf.predict(testArr)
relevance = results
relevance.shape

(166693,)

In [15]:
index = list(range(166694))[1:]

In [16]:
df = pd.DataFrame(index = index, data=relevance)
df

,0
1,2.33
2,2.33
3,3.00
4,3.00
5,3.00
6,2.33
7,3.00
8,3.00
9,3.00
10,3.00


In [772]:
df.to_csv("/Users/shijiabian/Dropbox/Duke Statistics Courses/Spring 2016 Duke Course/STA 571 Adv ML/Final Project/submission/sub_4_20.csv")

## April 23 Analysis

Get rid of the stop words from the product title and the description:

In [17]:
# add the two new columns: product title and the product description, no stop words involved
from nltk.corpus import stopwords
# ...
filtered_product_title = [word for word in result["product_title"] if word not in stopwords.words('english')]
result['filtered_product_title'] = pd.Series(filtered_product_title, index=result.index)

filtered_product_description = [word for word in result["product_description"] if word not in stopwords.words('english')]
result['filtered_product_description'] = pd.Series(filtered_product_description, index=result.index)

#product_description 

In [29]:
# add the two new columns: product title and the product description, no stop words involved
from nltk.corpus import stopwords
# ...
filtered_product_title = [word for word in result_test["product_title"] if word not in stopwords.words('english')]
result_test['filtered_product_title'] = pd.Series(filtered_product_title, index=result_test.index)

filtered_product_description = [word for word in result_test["product_description"] if word not in stopwords.words('english')]
result_test['filtered_product_description'] = pd.Series(filtered_product_description, index=result_test.index)

#product_description 

Get the word length of the product description and the product title

In [30]:
product_descr_len = result["filtered_product_description"].apply(split_search).apply(len)
#result["product_description"][1]
result['product_descr_len'] = pd.Series(product_descr_len, index=result.index)

product_title_len = result["filtered_product_title"].apply(split_search).apply(len)
#result["product_description"][1]
result['product_title_len'] = pd.Series(product_title_len, index=result.index)


In [31]:
#### for the test data
product_descr_len = result_test["filtered_product_description"].apply(split_search).apply(len)
#result["product_description"][1]
result_test['product_descr_len'] = pd.Series(product_descr_len, index=result_test.index)

product_title_len = result_test["filtered_product_title"].apply(split_search).apply(len)
#result["product_description"][1]
result_test['product_title_len'] = pd.Series(product_title_len, index=result_test.index)

The percentage of match for production description and product title

In [19]:
desc_match_perc = result["desc_match_word"]/result["product_descr_len"]
title_match_perc = result["title_match_word"]/result["product_title_len"]
result['desc_match_perc'] = pd.Series(desc_match_perc, index=result.index)
result['title_match_perc'] = pd.Series(title_match_perc, index=result.index)

In [32]:
# test data 
desc_match_perc = result_test["desc_match_word"]/result_test["product_descr_len"]
title_match_perc = result_test["title_match_word"]/result_test["product_title_len"]
result_test['desc_match_perc'] = pd.Series(desc_match_perc, index=result_test.index)
result_test['title_match_perc'] = pd.Series(title_match_perc, index=result_test.index)

In [33]:
from sklearn.ensemble import RandomForestClassifier
cols = ['desc_match_word', 'title_match_word', 'desc_match_perc', 'title_match_perc'] 
colsRes = ['relevance']
trainArr = result.as_matrix(cols) #training array
trainRes = result.as_matrix(colsRes) # training results
rf = RandomForestClassifier(n_estimators=100) # initialize
rf.fit(trainArr, trainRes) # fit the data to the algorithm


RandomForestClassifier(bootstrap=True, compute_importances=None,
            criterion='gini', max_depth=None, max_features='auto',
            max_leaf_nodes=None, min_density=None, min_samples_leaf=1,
            min_samples_split=2, n_estimators=100, n_jobs=1,
            oob_score=False, random_state=None, verbose=0)

In [34]:
testArr = result_test.as_matrix(cols)
results = rf.predict(testArr)
relevance = results
relevance.shape

df = pd.DataFrame(index = result_test['id'], data=relevance)
df


,0
id,
1,2.33
4,2.33
5,3.00
6,2.00
7,2.67
8,2.33
10,2.67
11,3.00
12,3.00


In [35]:
df.to_csv("/Users/shijiabian/Dropbox/Duke Statistics Courses/Spring 2016 Duke Course/STA 571 Adv ML/Final Project/submission/sub_4_20.csv")